# Generative AI Student Intervention

Phase 3: Business Heuristics & Generative AI Intervention
After our investigation in Phase 2, we discovered the "Cold Start Problem": it is nearly impossible to predict exam failure with high precision 14 or 7 days before the deadline because behavioral signals are not yet active (most students are dormant).
The Pivot: Instead of chasing a low-precision prediction, we apply the Reciprocal Heuristic: In education, the absence of evidence of progress is evidence of risk.
The Strategy: We segment students into three zones:
- Safe Zone (Green): Progress > 10%. (Action: None).
- Warning Zone (Yellow): Progress 1-9%. (Action: Low-cost nudge).
- Dormant Zone (Red): Progress == 0%. (Action: GenAI Intervention).
This notebook focuses on the Dormant Zone. We will use Google Gemini to generate hyper-personalized "re-engagement" plans for students with zero activity, helping them break their procrastination before it's too late.

In [30]:

import pandas as pd
# Import the official Google Generative AI library
import google.generativeai as genai
from google import genai
import os
import os
import time
from google import genai
from google.genai import types


In [ ]:
# Load API key from environment variable (from .env file)
import os

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found in environment. Set it in .env file")

client = genai.Client(api_key=GOOGLE_API_KEY)

In [32]:
print("Checking available models...")
for m in client.models.list():
    print(f"- {m.name}")

Checking available models...
- models/gemini-2.5-flash
- models/gemini-2.5-pro
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.5-flash-preview-tts
- models/gemini-2.5-pro-preview-tts
- models/gemma-3-1b-it
- models/gemma-3-4b-it
- models/gemma-3-12b-it
- models/gemma-3-27b-it
- models/gemma-3n-e4b-it
- models/gemma-3n-e2b-it
- models/gemini-flash-latest
- models/gemini-flash-lite-latest
- models/gemini-pro-latest
- models/gemini-2.5-flash-lite
- models/gemini-2.5-flash-image
- models/gemini-2.5-flash-lite-preview-09-2025
- models/gemini-3-pro-preview
- models/gemini-3-flash-preview
- models/gemini-3.1-pro-preview
- models/gemini-3.1-pro-preview-customtools
- models/gemini-3-pro-image-preview
- models/nano-banana-pro-preview
- models/gemini-3.1-flash-image-preview
- models/gemini-robotics-er-1.5-preview
- models/gemini-2.5-computer-use-preview-10-2025
- models/deep-research-pro-preview-12-2025
- 

In [33]:
MODEL_CANDIDATES = ["gemini-2.0-flash", "gemini-flash-latest", "gemini-pro-latest"]

In [34]:

# --- 2. Data Structures ---
# Using a dictionary for the student profile (Mocking the "Ghost Student" found via DuckDB)
student_profile = {
    "student_id": 43750,
    "days_to_exam": 14,
    "lessons_started": 0,
    "avg_progress": 0.0,
    "quizzes_attempted": 0
}

# --- 3. Prompt Engineering Logic ---
def get_intervention_prompt(profile: dict) -> str:
    """
    Constructs a high-context prompt for the LLM to generate a 
    personalized student intervention.
    """
    return f"""
    You are an empathetic, highly motivating Academic Success Coach at EnkelEksamen.
    Your goal is to help students pass their upcoming exams and prevent course repurchases.
    
    STUDENT CONTEXT:
    - Student ID: {profile['student_id']}
    - Days remaining: {profile['days_to_exam']}
    - Course progress: {profile['avg_progress']}%
    
    DIAGNOSIS:
    This is a "Ghost Student" (0% progress). They are likely paralyzed by procrastination.
    
    REQUIREMENTS:
    1. Tone: Warm, non-judgmental, yet encouraging.
    2. Action Plan: Suggest a 3-step plan for the first 48 hours to break the ice.
    3. Closing: Supportive and human.
    
    Generate the outreach email:
    """

# --- 4. Resilient Execution Logic ---
def run_student_intervention():
    """
    Attempts to generate the email using a fallback mechanism across 
    different model versions to ensure high availability.
    """
    prompt_text = get_intervention_prompt(student_profile)
    
    print(f"[LOG] Initiating GenAI connection for Student {student_profile['student_id']}...")
    
    for model_name in MODEL_CANDIDATES:
        try:
            print(f"[LOG] Attempting generation with model: {model_name}...")
            
            response = client.models.generate_content(
                model=model_name,
                contents=prompt_text
            )
            
            # If successful, print and exit loop
            print("\n" + "="*60)
            print(f"SUCCESS: INTERVENTION GENERATED ({model_name})")
            print("="*60 + "\n")
            print(response.text)
            return  # Exit function after success

        except Exception as e:
            print(f"[ERROR] Model {model_name} failed: {str(e)}")
            time.sleep(1)  # Brief pause before next attempt
            continue

    print("\n[CRITICAL] All model candidates failed. Check API Quota or Key status.")

# --- 5. Main Execution ---
if __name__ == "__main__":
    run_student_intervention()

[LOG] Initiating GenAI connection for Student 43750...
[LOG] Attempting generation with model: gemini-2.0-flash...
[ERROR] Model gemini-2.0-flash failed: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash\nPlease retry in 5.435455369s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'link